# Preprocessing videos for motion correction
This assumes that you have two tiff files with concatenated videos from a recording. One of them will be `raw` while to the other you should've applied a flat field subtraction (ffcSub). Also if you wnt to do any manual cropping to remove stable landmarks that hinder motion correction that should be done before starting processing here. 

This preprocessing code will apply FFT bandpass filter and other normalisation steps to your videos to help improve motion correction. The reason for doing this in python instead of imagej is to keep track of what was done and with what params by logging stuff. 

In [1]:
import os
import numpy as np

import logging
from fancylog import fancylog
import fancylog as package

from fcutils.file_io.io import load_yaml
from fcutils.file_io.utils import get_file_name
from utils import load_tiff_video_caiman

## Load metadata
You should have an `analysis_metadata.yml` file in your folder. This is used to keep trhack of what's what and what was done to which file. 

In [2]:
fld = 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05' # <- which folder/recording we are working on
metadata = load_yaml(os.path.join(fld, "01_PARAMS", "analysis_metadata.yml"))

# Start logging
logging_file = fancylog.start_logging(os.path.join(fld, "02_LOGS"), package, file_log_level="INFO", verbose=False, filename='preprocessing_logs')

# Write down metadata about analysis
logging.info("ANALYSIS METADATA FILE:")
for k,v in metadata.items():
    logging.info(f"{k}:  {v}")

2020-04-17 17:02:31 PM - INFO - MainProcess fancylog.py:271 - Starting logging
2020-04-17 17:02:31 PM - INFO - MainProcess fancylog.py:273 - Multiprocessing-logging module not found, not logging multiple processes.
2020-04-17 17:02:31 PM - INFO - MainProcess <ipython-input-2-1ffcacef5ea6>:8 - ANALYSIS METADATA FILE:
2020-04-17 17:02:31 PM - INFO - MainProcess <ipython-input-2-1ffcacef5ea6>:10 - experimenter:  Federico
2020-04-17 17:02:31 PM - INFO - MainProcess <ipython-input-2-1ffcacef5ea6>:10 - fld:  D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05
2020-04-17 17:02:31 PM - INFO - MainProcess <ipython-input-2-1ffcacef5ea6>:10 - raw_video:  19JUN05_BF164p1_v1_ds126_crop_raw.tif
2020-04-17 17:02:31 PM - INFO - MainProcess <ipython-input-2-1ffcacef5ea6>:10 - ffcsub_video:  19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff
2020-04-17 17:02:31 PM - INFO - MainProcess <ipython-input-2-1ffcacef5ea6>:10 - ffcsub_params:  {'sigma': 25, 'mean': 'each_frames_video'}
2020-04-17 17:02

## Load video
Loading the ffcsub video for further preprocessing

In [3]:
video = load_tiff_video_caiman(os.path.join(fld, metadata['ffcsub_video']))
print(f"Loaded video with {video.shape[0]} frames and frame size: [{video.shape[1]}, {video.shape[2]}]")


Loaded video with 22662 frames and frame size: [109, 92]


## Apply FFT bandpass filter
Apply the filter to get a filtered movie (essentially removing high freq noise) and use it to normalise the ffcsub by division. 

I coudn't get imagej to work in python, so for now this stepp will be done externally in Fiji. 
After you've filtered the video, please update the metadata file to specify where the output was saved and what params you've used for the filtering. Then reload the the metadata

For a description of what the FFT filter in fiji does: https://imagej.nih.gov/ij/plugins/fft-filter.html

In [4]:
# When done in Fiji, update the params and load the video
metadata = load_yaml(os.path.join(fld, "01_PARAMS", "analysis_metadata.yml"))

logging.info(f"fft bandpass parameters: {metadata['fft_bandpass_params']}")
logging.info(f"fft bandpass video saved at: {metadata['fftfilt_video']}")

fftfilt_video = load_tiff_video_caiman(os.path.join(fld, metadata['fftfilt_video']))
print(f"Loaded video with {fftfilt_video.shape[0]} frames and frame size: [{fftfilt_video.shape[1]}, {fftfilt_video.shape[2]}]")


2020-04-17 17:02:33 PM - INFO - MainProcess <ipython-input-4-ac1fe74475ac>:4 - fft bandpass parameters: [{'large_structures': 100}, {'small_structures': 5}, {'suppress_stripes': 'None'}, {'autoscale': True}, {'saturate': False}, {'display': False}]
2020-04-17 17:02:33 PM - INFO - MainProcess <ipython-input-4-ac1fe74475ac>:5 - fft bandpass video saved at: 19JUN05_BF164p1_v1_ds126_crop_ffcSub_fftfilt.tif
Loaded video with 22662 frames and frame size: [109, 92]


## Normalize ffcsub using fft bandpass filtered video
Divide video by fftfilt video

In [5]:
# Check that neither of the two videos has zeros
if np.min(video)  <= 0 or np.min(fftfilt_video) <= 0:
    raise ValueError("The videos can't have 0 for this step, we'll be dividing stuff!")

# Divide and save
video_divided_by_fft = video / fftfilt_video

savename = get_file_name(metadata['ffcsub_video'])+"_div_fft.tif"
savepath = os.path.join(fld, savename)
video_divided_by_fft.save(savepath)

logging.info(f"Saved video normalized by fft filtered at: {savepath}")
logging.info("Done.")

print("\nDon't forget to add to the metadata file the name of the video normalised by fft")


2020-04-17 17:02:38 PM - INFO - MainProcess <ipython-input-5-5ecff2e7115b>:12 - Saved video normalized by fft filtered at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft.tif
2020-04-17 17:02:38 PM - INFO - MainProcess <ipython-input-5-5ecff2e7115b>:13 - Done.

Don't forget to add to the metadata file the name of the video normalised by fft
